**Import all the libraries and modules required.**

In [1]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

**Build the neural network**

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

**Image Data Generation/Augmentation**

In [3]:
TRAINING_DIR = "/content/drive/MyDrive/Dataset/train/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/drive/MyDrive/Dataset/test/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1327 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


**Saving best model**

In [7]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

**Train the model**

In [8]:
history = model.fit_generator(train_generator,
                              epochs=100,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
133/133 [==============================] - 737s 6s/step - loss: 0.5535 - acc: 0.7280 - val_loss: 0.1775 - val_acc: 0.9433
INFO:tensorflow:Assets written to: model2-001.model/assets
Epoch 2/100
133/133 [==============================] - 20s 148ms/step - loss: 0.3217 - acc: 0.8734 - val_loss: 0.2490 - val_acc: 0.9742
Epoch 3/100
133/133 [==============================] - 19s 144ms/step - loss: 0.2971 - acc: 0.8855 - val_loss: 0.1026 - val_acc: 0.9588
INFO:tensorflow:Assets written to: model2-003.model/assets
Epoch 4/100
133/133 [==============================] - 19s 145ms/step - loss: 0.2271 - acc: 0.9171 - val_loss: 0.0696 - val_acc: 0.9845
INFO:tensorflow:Assets written to: model2-004.model/assets
Epoch 5/100
133/133 [==============================] - 19s 145ms/step - loss: 0.2433 - acc: 0.8975 - val_loss: 0.1894 - val_acc: 0.8969
Epoch 6/100
133/133 [==============================] - 19s 144ms/step - loss: 0.2357 - acc: 0.9179 - val_loss: 0.0835 - val_acc: 0.9691
Epoch 7/1

**Download the best Model checkpoint**

In [ ]:
!zip -r /content/BestModel.zip /content/model2-077.model

In [ ]:
from google.colab import files
files.download('model2-077.model') 

**TEST**

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("/content/model2-010.model")

In [ ]:
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
# cap = cv2.VideoCapture(0) 

haarcascade = cv2.CascadeClassifier('/content/Haar/haarcascade_frontalface_default.xml')
while True:
    (rval, im) = Image(filename)
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()